In [118]:
import os
import glob
import os.path
import ast
import sys
import json
import time
import requests
import openpyxl
import datetime
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import SessionNotCreatedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException

### 동적 웹 크롤링

#### 맛집 블로그 포스트 100개 크롤링 - 제목, url, 장소명

In [119]:
def GetPostDate(post_title):        
    # 검색된 포스트의 날짜 구하기 - Naver API 활용
    client_id = "XVCwmN8OWSwXTVWc2_uQ"
    client_secret = "45zos8jb_F"

    encText = urllib.parse.quote(str(post_title))

    url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
    display_num = "1" #출력할 갯수 입력받기
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display="+display_num# json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read().decode('utf-8')
        # print(response_body)
        # json 파일로 변경
        with open("response.json", "w") as json_file:
            json.dump(response_body, json_file)
        res_json = json.dumps(response_body, indent=4, sort_keys=True)
        with open("response.json", "r") as response_json:
            res_python = json.load(response_json)

        #items key 값만 뽑기
        my_dict = ast.literal_eval(res_python)
        list_str = str(my_dict["items"])[1:-1]
        # json 파일로 변경
        with open("items.json", "w") as json_file:
            json.dump(list_str, json_file)
        res_json = json.dumps(list_str, indent=4, sort_keys=True)
        with open("items.json", "r") as response_json:
            res_python = json.load(response_json)
        my_dict = ast.literal_eval(res_python)
        postdate = my_dict["postdate"]
        # print("postdate : {}".format(postdate))
    else:
        print("Error Code:" + rescode)
    return postdate

In [120]:
def GetResInfo(raw_url):

    temp = []
    tags = []

    #파싱 가능한 실제 viewer url 얻기
    # print(raw_url)
    url = "view-source:{}".format(raw_url)
    # print(url)

    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))

    driver.get(url)
    time.sleep(3)
    a = driver.find_element(By.CSS_SELECTOR, 'body > table > tbody > tr:nth-child(76) > td.line-content > a').text
    # print(a)
    real_url = "https://blog.naver.com{}".format(a)
    
    #파싱 가능한 url로 재진입
    driver.get(real_url)
    time.sleep(3)
    print(real_url)

    #식당명, 식당주소 얻기
    page = urllib.request.urlopen(real_url)
    soup = BeautifulSoup(page,'html.parser')
    try:
        # for tag in soup.find_all(class_='se-map-title'):
        for tag in soup.find(class_='se-map-title'):
            restaurant_name = tag.get_text()
            print(restaurant_name)
        # for tag in soup.find_all(class_='se-map-address'):
        for tag in soup.find(class_='se-map-address'):
            restaurant_adress = tag.get_text()
            print(restaurant_adress)
    except TypeError as e:
        restaurant_name = ""
        restaurant_adress = ""
        print("본문내 지도 위젯이 없습니다.")

    # 해시태그 확인용 url 재진입
    # 해시태그 얻기    
    # b = driver.find_element(By.CSS_SELECTOR, '#post_footer_contents').text
    # str(b).split('\n')
    # temp = list(b.split('\n'))
    # tags = temp[4:-1]
    # print(tags)

    #해시태그 얻기_2024.08.26 수정
    b = driver.find_element(By.CSS_SELECTOR, '#post_footer_contents').text
    temp = list(str(b).split('\n'))
    tags = temp[1:]
    if '영리적 사용 불가' in tags:
        tags.remove('영리적 사용 불가')
    if '내용 변경 불가' in tags:
        tags.remove('내용 변경 불가')
    if '태그' in tags:
        tags.remove('태그')
    tags_ = ', '.join(tags)
    print(tags)
    print(tags_)

    time.sleep(3)
    driver.close()
    
    return str(restaurant_name), str(restaurant_adress), str(tags_)

In [121]:
def GetResInfo_old(url):
    print(url)
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
        
    #블로그 링크 하나씩 불러서 크롤링
    tags = []
    contents = []
    restaurant_name = []
    restaurant_address = []
    
    #블로그 링크 하나씩 불러오기
    driver.get(url)
    time.sleep(1)
    #블로그 안 본문이 있는 iframe에 접근하기
    driver.switch_to.frame("mainFrame")

    #본문 전체
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        contents.append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        contents.append(a)
    #print(본문: \n', a)

    #해시태그
    # try:
    #     t = driver.find_element(By.CSS_SELECTOR,'div.post-btn post_btn2')
    #     tags.append(t)
    # except NoSuchElementException:
    #     print("해시태그를 찾을 수 없습니다.")

    # driver.quit() #창닫기
    print("<<본문 크롤링이 완료되었습니다.>>")
    text = contents[0]
    contents_list = text.split("\n")
    print(contents_list)

    print("<<본문의 장소 정보입니다.>>")
    try:
        found = contents_list.index('© NAVER Corp.') ## 네이버 지도 전체가 위젯으로 넣어진 경우
        restaurant_name.append(contents_list[found+1])
        restaurant_address.append(contents_list[found+2])
        print(restaurant_name)
        print(restaurant_address)
    except ValueError:
        try : 
            found = contents_list.index('예약')
            restaurant_name.append(contents_list[found-2]) ##네이버 지도 일부가 위젯으로 넣어지거나, 
            restaurant_address.append(contents_list[found-1])
            print(restaurant_name)
            print(restaurant_address)
        except ValueError:
            try : 
                found = contents_list.index('주소')
                restaurant_name.append(contents_list[found-1]) ##지도가 본문에 첨부되지 않은 경우
                restaurant_address.append(contents_list[found+1])
                print(restaurant_name)
                print(restaurant_address)
            except ValueError:
                print("장소 정보를 찾을 수 없습니다.")
                restaurant_name.append(" ")
                restaurant_address.append(" ")
                print(restaurant_name)
                print(restaurant_address)
        
    try:
        tag = [s for s in contents_list if "#" in s] 
        print(tag)
        tags.append(tag)
    except IndexError:
            print("해시태그를 찾을 수 없습니다.")

    return restaurant_name[0], restaurant_address[0], str(tags)
    driver.quit() #창닫기

In [122]:
def GetNaverBlog(url, crawler_date, crawler_time):
    App = "Naver Blog"
    data = []

    #크롬 드라이버 연결
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
    driver.get(url)
    time.sleep(3)
    
    # post_titles = driver.find_elements(By.CLASS_NAME,'title_post')

    #크롤링 시작
    print("-----네이버 블로그 - 맛집 포스트-----")
    for page in range(1,2): ##1~n-1 페이지까지. 5분 소요
        driver.find_element(By.XPATH, '/html/body/ui-view/div/main/div[1]/div/section/div[2]/div[11]/span[{}]/a'.format(page)).click()
        time.sleep(2)

        #포스트 제목
        post_titles = driver.find_elements(By.CLASS_NAME,'title_post')

        #포스트 url, 식당명, 식당주소
        post_urls = driver.find_elements(By.CLASS_NAME,'desc_inner')
        urls = []
        # post_dates = []
        name = []
        address = []
        tags = []

        for i in post_urls :
            href = i.get_attribute('href')
            # print(href)
            urls.append(href)
        # print(urls)
        
        for url in urls :
            res_name, res_address, tag = GetResInfo(url)
            name.append(res_name)
            address.append(res_address)
            tags.append(tag)

        # for post_title in post_titles:
        #     print(post_title)
        #     post_date = GetPostDate(post_title)
        #     post_dates.append(post_date)
            

        for idx, post_title in enumerate(post_titles[9:]): ## Top 3 제거(제목너무 짧음), 공백 행 제거
            title = post_title.text
            line_data = []
            line_data.append(App)
            line_data.append(GetPostDate(title))
            line_data.append(title)
            line_data.append(urls[idx])
            line_data.append(name[idx])
            line_data.append(address[idx])
            line_data.append(tags[idx])
            line_data.append(crawler_time)
            data.append(line_data)
        
    #데이터 프레임 생성
    df = pd.DataFrame(data, columns=['App', 'Post Date', 'Post Title', 'URL', 'Restaurant Name', 'Restaurant Address', 'Tag', 'Crawler Time'])
    df.index = df.index+1
    df

    #엑셀로 저장
    df.to_excel('Results/FD_Crawling_Naver_Blog_{}.xlsx'.format(crawler_date), index=False)
    time.sleep(3)
    driver.close()

#### 공정위 가맹사업정보제공시스템 정보공개서 크롤링 - 100개

In [123]:
def GetBusinessInfoSystem (url, crawler_date, crawler_time):
    
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
    driver.get(url)
    time.sleep(3)

    #외식 업종, 100개씩 보기, 적용 조회
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[1]/fieldset/select[2]/option[2]').click()
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[1]/fieldset/input[2]').click()
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[2]/div/select/option[4]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[2]/div/a').click() #적용 클릭
    time.sleep(3)
    table = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/table')
    tbody = table.find_element(By.TAG_NAME, 'tbody')

    i = 1
    data = []
    line_data = []
    
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["No.", "Name", "Business", "Representative","Registration_Number", "Init_Date","Type"])
    # 번호, 상호, 영업표지, 대표자, 등록번호, 최초등록일, 업종

    rows = tbody.find_elements(By.TAG_NAME, "tr")

    #테이블 파싱, 컬럼명 지정
    for index, value in enumerate(rows):
        Name = value.find_elements(By.TAG_NAME,"td")[1].text
        Business = value.find_elements(By.TAG_NAME,"td")[2].text
        Representative = value.find_elements(By.TAG_NAME,"td")[3].text
        Registration_Number = value.find_elements(By.TAG_NAME,"td")[4].text
        Init_Date = value.find_elements(By.TAG_NAME,"td")[5].text
        Type = value.find_elements(By.TAG_NAME,"td")[6].text

        sheet.append([i, Name, Business, Representative, Registration_Number, Init_Date, Type])
        i +=1

    for row in sheet.iter_rows(max_col=7, values_only=True):
        print(row)
        line_data.append(row)
        data.append(line_data)

    #saving to excel
    # 정렬 전 저장
    wb.save('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date))

    # 정렬 후 저장
    xl = pd.ExcelFile('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date))
    df = xl.parse("Sheet")
    df = df.sort_values(by="Init_Date", ascending=False)
    df.to_excel('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date), sheet_name="가맹사업정보", columns=["No.", "Name", "Business", "Representative","Registration_Number", "Init_Date","Type"],index=False)
    driver.close()

In [124]:
def merge_excel(file_path, total_file, arch_folder):

    if os.path.isfile(total_file) :
        print("기준 엑셀 파일에 병합합니다...")
        df1 = pd.read_excel(file_path)
        df2 = pd.read_excel(total_file)
        
        df_total = pd.concat([df1,df2], ignore_index=False)
        df_total.drop_duplicates(subset=None, keep='first',inplace=False, ignore_index=False)
        df_total = df_total.drop('No.',axis=1)

    else:
        print("파일이 존재하지 않습니다.")
    
    # print(df_total)
    df_total.to_excel('./results/FD_Crawling_Business_Info_System_Total.xlsx', index=False)
    print("병합을 완료하고 해당 파일을 보관합니다...")
    
    import shutil
    # 이미 존재하면 덮어 쓰는 조건문 만들기
    shutil.move(file_path, arch_folder)

In [129]:
def main():

    # 네이버 맛집 블로그 콘텐츠 크롤링
    crawler_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    print("Naver crawler_date : {}".format(crawler_time))

    naver_url = "https://section.blog.naver.com/ThemePost.naver?directoryNo=29&activeDirectorySeq=3&currentPage=1" #네이버 블로그 맛집 카테고리 메인 화면
    GetNaverBlog(naver_url, crawler_date, crawler_time)

    # 가맹사업정보시스템 크롤링
    crawler_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    print("가맹사업정보시스템 crawler_date : {}".format(crawler_time))

    business_info_system_url = "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do"
    GetBusinessInfoSystem(business_info_system_url, crawler_date, crawler_time)

    file_list = glob.glob('./results/FD_Crawling_Business_Info_System_2*.xlsx')
    total_file = './results/FD_Crawling_Business_Info_System_Total.xlsx'
    arch_folder = './results/FD_Crawling_Business_Info_System_Archaive'

    for file_path in file_list:
        print(file_path)
        print(total_file)
        merge_excel(file_path, total_file,arch_folder)

In [130]:
if __name__ == "__main__":
    main()

./results\FD_Crawling_Business_Info_System_20240827.xlsx
./results/FD_Crawling_Business_Info_System_Total.xlsx
기준 엑셀 파일에 병합합니다...
병합을 완료하고 해당 파일을 보관합니다...
